# Before you start :
   - These exercises are related to the Pivot table and correlation lessons.
   - Keep in mind that you need to use some of the functions you learned in the previous lessons.
   - All datasets are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

#### Import all the libraries that are necessary.

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 

#### Open the ` Employee.csv` file and name your dataset `employee`.

In [63]:
employee = pd.read_csv('Employee.csv')
employee.head()

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30
3,Sonia,HR,Bachelor,F,analyst,4,35
4,Samuel,Sales,Master,M,associate,3,55


#### What's the mean salary by department?
Hint : There are 2 ways to do it - one way is with groupby and the other way is with pivot tables. Do it both ways. 


In [16]:
# Method 1

employee.groupby(['Department'])['Salary'].agg(np.mean)


Department
HR       45.00
IT       48.75
Sales    55.00
Name: Salary, dtype: float64

#### We want now to group the data by more than one column. Use Pandas pivot table function to select the mean salary by department and title.

In [44]:
# Method 2

employee.pivot_table(index = ['Department', 'Title'], values = ['Salary'], aggfunc = [np.mean]).reset_index()


,Department,Title,mean
,,,Salary
0,HR,VP,70.0
1,HR,analyst,32.5
2,IT,VP,70.0
3,IT,analyst,32.5
4,IT,associate,60.0
5,Sales,associate,55.0


#### We want to display the number of years of work experience and the salary for each Title. Add Years as a column in your pivot table.
*Hint: Use Pandas doc(see References in README.md) to explore the pivot_table() arguments*

In [49]:
employee.pivot_table(index = ['Title'], values = ['Years','Salary']).reset_index()

,Title,Salary,Years
0,VP,70.000000,8.00
1,analyst,32.500000,2.25
2,associate,56.666667,4.00


#### We want to know the information about the mean salary and number of employees in each department and for each job title.

In [74]:
# employee.pivot_table(index = ['Title'], values = ['Salary', 'Department'], aggfunc = {'Salary':np.mean, 'Department':'count'}).reset_index()

# The above method only works if we create a new column for Department aggregated by count (so we don't get an error for having two indexes)

# Groupby method works best here:

employee.groupby(['Department','Title']).agg({'Salary': np.mean, 'Department':'count'})

Salary  Department
Department Title                        
HR         VP           70.0           1
           analyst      32.5           2
IT         VP           70.0           1
           analyst      32.5           2
           associate    60.0           1
Sales      associate    55.0           2

#### We want to know for each department and for each job title the Salary's median, minimum, maximum and standard deviation. Comment your results.

In [84]:
employee_salary_metrics = employee.groupby(['Title', 'Department']).agg({'Salary': [np.median, min, max, np.std]})

employee_salary_metrics.columns = ['Salary_median', 'Salary_min', 'Salary_max', 'Salary_std']

employee_salary_metrics # We can see that there are NaN values in the std column

Salary_median  Salary_min  Salary_max  Salary_std
Title     Department                                                   
VP        HR                   70.0          70          70         NaN
          IT                   70.0          70          70         NaN
analyst   HR                   32.5          30          35    3.535534
          IT                   32.5          30          35    3.535534
associate IT                   60.0          60          60         NaN
          Sales                55.0          55          55    0.000000

#### Based on your comments, fill the missing values with an appropriate value.

In [86]:
# Replace the NaN values with 0 because that would be the correct std based on the descriptive statistics 

# (i.e., there is only a single observation for the values: VPs of HR and IT, hence we cannot calculate a std)

employee_salary_metrics['Salary_std'] = employee_salary_metrics['Salary_std'].fillna(0)

employee_salary_metrics

Salary_median  Salary_min  Salary_max  Salary_std
Title     Department                                                   
VP        HR                   70.0          70          70    0.000000
          IT                   70.0          70          70    0.000000
analyst   HR                   32.5          30          35    3.535534
          IT                   32.5          30          35    3.535534
associate IT                   60.0          60          60    0.000000
          Sales                55.0          55          55    0.000000

#### The stake holders want to know for each department the number of employees and how much money is spend on salaries. Could you provide that information? 

In [89]:
employee_salary_sums = employee.groupby(['Department']).agg({'Salary': sum, 'Department':'count'})

employee_salary_sums.columns = ['Salary_Total_per_Dept', 'Personnel']

employee_salary_sums

,Salary_Total_per_Dept,Personnel
Department,,
HR,135,3
IT,195,4
Sales,110,2


#### For each Department and Title, we want to know the the total years of work experience and the mean salary.

In [91]:
employee_experience = employee.groupby(['Department', 'Title']).agg({'Years': sum, 'Salary': np.mean})

employee_experience.columns = ['Aggregate_Years_Exp', 'Avg_Salary']

employee_experience

Aggregate_Years_Exp  Avg_Salary
Department Title                                     
HR         VP                           8        70.0
           analyst                      6        32.5
IT         VP                           8        70.0
           analyst                      3        32.5
           associate                    7        60.0
Sales      associate                    5        55.0

### Bonus

#### We now want to compute the mean salary after removing the maxima for each Department. Create the appropriate pivot table.

Hint: Write a custom function.